In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set(color_codes=True)

import matplotlib.pyplot as plt

%matplotlib inline
%precision 2

In [ ]:
data_df = pd.read_csv('../input/dont-overfit-ii/train.csv')

data_df.sample(3)

In [ ]:
data_df['target'].value_counts().plot(kind='bar')
plt.xlabel('label')
plt.title('Distribution of classes among the training data')

## Distribution of first 24 columns

In [ ]:
plt.figure(figsize=(26, 24))
for i, col in enumerate(data_df.columns[2:26]):
    plt.subplot(6, 4, i + 1)
    sns.distplot(data_df[col])

In [ ]:
data_df[data_df.columns[2:]].mean().plot('hist')
plt.title('Distribution of Mean of each column');

In [ ]:
plt.figure(figsize=(8, 5))
for i, col in enumerate(data_df.columns[2:6]):
    plt.subplot(2, 2, i + 1)
    sns.violinplot(data=data_df, y=col, x='target')

In [ ]:
X = data_df.drop(columns=["target", "id"])
y = data_df["target"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Random Forest
Fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

The sub-sample size is always the same as the original input sample size but the samples are drawn with replacement if bootstrap=True (default).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=2, bootstrap=True)
rfc.fit(X_train, y_train)
# make predictions using the trained model
y_pred = rfc.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred, y_test))

from sklearn.metrics import accuracy_score
print('accuracy is ',accuracy_score(y_pred, y_test))

## Meta Analysis
A meta-analysis is a statistical analysis that combines the results of multiple scientific studies. Meta-analysis can be performed when there are multiple scientific studies addressing the same question, with each individual study reporting measurements that are expected to have some degree of error. 

### Bagging Classifier

An ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag_model = BaggingClassifier()
bag_model.fit(X_train, y_train)
y_pred=bag_model.predict(X_test)

print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print('accuracy is ',accuracy_score(y_pred, y_test))

### AdaBoost
An AdaBoost classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
abc.fit(X_train, y_train)
y_pred = abc.predict(X_test)

print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print('accuracy is ',accuracy_score(y_pred, y_test))

### Gradient Boosting Classifiers
GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)

print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print('accuracy is ',accuracy_score(y_pred, y_test))

### LDA/QDA
- Linear Discriminant Analysis -  a method used in statistics, pattern recognition, and machine learning to find a linear combination of features that characterizes or separates two or more classes of objects or events.

- Qudratic Discriminatn Analysis - Here the combination is not linear in nature.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)

print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print('accuracy is ',accuracy_score(y_pred, y_test))

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred = qda.predict(X_test)

print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred, y_test))
print('accuracy is ',accuracy_score(y_pred, y_test))

## Permutation Importance of each feautre
Permutation importance is calculated after a model has been fitted. It shows how much does a feature effect the predictions. It answers the following question.
_If we randomly shuffle a single column of the validation data, leaving the target and all other columns in place, how would that affect the accuracy of predictions in that now-shuffled data?_

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)

rfc_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(test_X, test_y)
eli5.show_weights(perm, feature_names=test_X.columns.tolist(), top=300)